[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_load_patterns/EMQA_load_patterns.ipynb)

# EMQA_load_patterns

Electricity load patterns analysis.

**Output:** `load_patterns.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import os

def load_romania():
    paths = [
        '../../charts/ro_de_prices_full.csv',
        '/Users/danielpele/Documents/Energy MBA/charts/ro_de_prices_full.csv',
    ]
    for p in paths:
        if os.path.exists(p):
            return pd.read_csv(p, parse_dates=['date'], index_col='date')
    raise FileNotFoundError("ro_de_prices_full.csv not found")

ro = load_romania()


In [ ]:
consumption = ro['ro_consumption'] / 1000  # MW to GW

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

daily = consumption.groupby(consumption.index.dayofweek).mean()
days = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
axes[0].bar(range(7), daily.values, color=[COLORS['blue']]*5+[COLORS['orange']]*2, alpha=0.7)
axes[0].set_xticks(range(7)); axes[0].set_xticklabels(days)
axes[0].set_title('Weekly Load Pattern', fontweight='bold')
axes[0].set_ylabel('GW')

monthly = consumption.groupby(consumption.index.month).mean()
months = ['J','F','M','A','M','J','J','A','S','O','N','D']
colors_m = [COLORS['blue'] if m in [12,1,2] else COLORS['red'] if m in [6,7,8] else COLORS['gray'] for m in range(1,13)]
axes[1].bar(range(12), monthly.values, color=colors_m, alpha=0.7)
axes[1].set_xticks(range(12)); axes[1].set_xticklabels(months)
axes[1].set_title('Seasonal Load Pattern', fontweight='bold')
axes[1].set_ylabel('GW')

if 'ro_temp_mean' in ro.columns:
    axes[2].scatter(ro['ro_temp_mean'], consumption, s=5, alpha=0.3, color=COLORS['purple'])
    axes[2].set_xlabel('Temperature (C)'); axes[2].set_ylabel('GW')
    corr = consumption.corr(ro['ro_temp_mean'])
    axes[2].set_title(f'Load vs Temperature (r={corr:.2f})', fontweight='bold')
else:
    axes[2].plot(consumption.index, consumption, color=COLORS['blue'], lw=0.5)
    axes[2].set_title('Consumption Time Series', fontweight='bold')

fig.suptitle('Romanian Electricity Load Patterns', fontweight='bold', fontsize=13)
fig.tight_layout(rect=[0,0,1,0.95])
save_fig(fig, 'load_patterns.pdf')
plt.show()
